In [1]:
import numpy as np
from itertools import combinations, chain, permutations, product
from functools import partial
from itertools import combinations_with_replacement as cwr
from math import factorial as bang
from scipy.misc import comb
from imagery_psychophysics.src.stirling_maps import stirling_partitions as stp
from imagery_psychophysics.src.stirling_maps import stirling_num_of_2nd_kind as snk
from time import time

%matplotlib inline

* To do
    - separate methods for selection / partitioning / assignment
    - implement recursive counting of selection component -- check against known formula
    - implement recursive counting of all legal maps -- check against known formula

#### helpful functions

In [2]:
##powerset of a list
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

##all strict subsets
def strict(iterable):
    pwr = set(powerset(iterable))
    pwr.discard(tuple(iterable))
    return pwr


##all strict, non-empty subsets
def strictnonempty(iterable):
    pwr = set(powerset(iterable))
    pwr.discard(tuple(iterable))
    pwr.discard(())
    return pwr
    


##all subsets of the integers -- redundant.
allsubsets = lambda n: list(chain(*[combinations(range(n), ni) for ni in range(n+1)]))

##nonempty subsets of the integers up to n
def nonemptysubsets(n):
    allsubs = set(allsubsets(n))
    allsubs.discard(())
    return allsubs

##unique integers from keys
def integers_from_keys(mydict):
    return list(np.unique(sum(mydict.keys(),())))  ## == [m]

##return subset of a dictionary with certain keys ignored
def ignore_keys(ignore_any_key_with, mydict):
    return {key: mydict[key] for key in mydict.keys() if all([x not in key for x in ignore_any_key_with])}
    

#### construct an intersect-complement-union partion using indexed basis sets: a basis set will be the number of probe windows whose unions are measured during the experiment. so if the number of basis sets is four, we will acquire subject responses to all possible unions of those four windows, including the "singletons" where the windows are shown by themselves.

In [10]:
##measure the cardinality of each block of "Venn diagram" partition
def block_size(union_subset, union_value_dict, size_of_universe = 0):
    '''notation guide:
    union_subset = S
    union_value_dict[T] = d_T
    '''
    doPrint = 1
    basis_set_indices = integers_from_keys(union_value_dict)
 
    adder = 0
    if len(union_subset) == len(basis_set_indices):
        return size_of_universe - union_value_dict[union_subset]
    for F in powerset(union_subset):
        sgn = len(union_subset)-len(F)
        if doPrint:
            print 'F |-------> %s' %(F,)
        for T in powerset(set(basis_set_indices)-set(F)):
            if T:
                if doPrint:
                    print 'T |--> %s' %(T,)
                adder += (-1)**(sgn+len(T)+1)*union_value_dict[T]
                if doPrint:
                    print 'adder: %d' %(adder)
    return adder



In [15]:
##simple example
number_of_basis_sets = 2
S = (0,)
union_value_dict = {}
for ci in allsubsets(number_of_basis_sets):
    union_value_dict[ci] = 1
# union_value_dict[()] = 0
print union_value_dict
print block_size(S, union_value_dict, size_of_universe=2)

{(0, 1): 1, (): 1, (1,): 1, (0,): 1}
F |-------> ()
T |--> (0,)
adder: -1
T |--> (1,)
adder: -2
T |--> (0, 1)
adder: -1
F |-------> (0,)
T |--> (1,)
adder: 0
0


#### count the number of sets possible by building a set from the blocks of a partition. For each block of a partition, we intersect the target set with the blocks of the partition. If the block contains n elements, and the intersection of the target with the block contains k elements, then there are n-choose-k ways of constructing that little compartment of the target set.

In [19]:
def count_sets(target_index, union_value_dict, size_of_universe = 0):
    basis_set_indices = list(set(integers_from_keys(union_value_dict)) - set((target_index,)))  ##these are the basis sets
    basis_values_only = ignore_keys((target_index,), union_value_dict)
    output = 1
    doPrint = 0
    for S in powerset(basis_set_indices):
        if doPrint:
            print 'S |-------------> %s' %(S,)
        upper = block_size(S, basis_values_only,size_of_universe)
        if doPrint:
            print 'S |-------------> %s' %(S,)
        downer = block_size(S , union_value_dict,size_of_universe)
        print upper
        print downer
        output *= comb(upper, downer)
#         print output
    
    return output

In [20]:
##specific example

##set_1 is the basis set, set_0 is the target set. Here we list the sizes of the sets and their union
union_value_dict = {(0, 1, 2): 1, (0,): 1, (1,): 1, (2,): 1, (1,2): 1, (0,1): 1, (0,2): 1} 
print union_value_dict 

##how many target map
target_index=0
count_sets(target_index, union_value_dict,size_of_universe=1)

{(1, 2): 1, (0, 1): 1, (0,): 1, (1,): 1, (0, 1, 2): 1, (2,): 1, (0, 2): 1}
1
1
0
0
0
0
0
0


1.0

In [21]:
##this will help count 
def number_of_basis_interactions(number_of_basis_sets):
    return 2**number_of_basis_sets-1

In [22]:
print number_of_basis_interactions(7)

127


In [23]:
for jj in range(2,10+1):
    for ii in range(2,5+1):
        print '(%d objects and %d basis sets) gives (%d interactions and %d potential combos'         %(jj, 
                                                                                                        ii,
                                                                                                        number_of_basis_interactions(ii),
                                                                                                        jj**number_of_basis_interactions(ii))


(2 objects and 2 basis sets) gives (3 interactions and 8 potential combos
(2 objects and 3 basis sets) gives (7 interactions and 128 potential combos
(2 objects and 4 basis sets) gives (15 interactions and 32768 potential combos
(2 objects and 5 basis sets) gives (31 interactions and 2147483648 potential combos
(3 objects and 2 basis sets) gives (3 interactions and 27 potential combos
(3 objects and 3 basis sets) gives (7 interactions and 2187 potential combos
(3 objects and 4 basis sets) gives (15 interactions and 14348907 potential combos
(3 objects and 5 basis sets) gives (31 interactions and 617673396283947 potential combos
(4 objects and 2 basis sets) gives (3 interactions and 64 potential combos
(4 objects and 3 basis sets) gives (7 interactions and 16384 potential combos
(4 objects and 4 basis sets) gives (15 interactions and 1073741824 potential combos
(4 objects and 5 basis sets) gives (31 interactions and 4611686018427387904 potential combos
(5 objects and 2 basis sets) gives

In [24]:
def enumerate_legal_combos(number_of_objects=1, number_of_basis_sets=2):
    doprint = 0
    ##nobs = number of basis sets
    ##nobj = number of objects (size of universe)
    nobj = number_of_objects
    nobs = number_of_basis_sets
    total_possible_combos = nobj**number_of_basis_interactions(nobs)
    print 'total of %d possible combos' %(total_possible_combos)
    all_unions_of_basis_sets = list(nonemptysubsets(nobs)) ##[(1,), (2,), ... , (1,2), ... , (1,nobs), ... , (1,2,...,nobs)]
    all_unions_of_basis_sets.sort(key=lambda t: len(t), reverse=False) ##sorts by length of tuples
    all_union_dict = dict(zip(all_unions_of_basis_sets, range(len(all_unions_of_basis_sets))))
    parents_for_union_floor = lambda cur_union: map(lambda y: all_union_dict[y], combinations(cur_union,len(cur_union)-1))
    get_min = lambda cur_union,val,mapping_func = parents_for_union_floor: max(val[mapping_func(cur_union)]) ##
    get_max = lambda cur_union, val: min(nobj, min(map(lambda z: np.sum(val[z]), map(lambda y: [all_union_dict[y[0]], all_union_dict[y[1]]], [map(tuple,ii) for ii in list(stp(cur_union,2))]))))


    ##for singletons, all possible combinations of values between 1 and nobs are legal
    all_values_of_unions = list(product(range(1,nobj+1), repeat = nobs))  ##[(1,1,1...), (1,1,nobs,...), etc]
    if doprint:
        print 'all unions: %s' %(all_unions_of_basis_sets)
        print 'start values: %s' %(all_values_of_unions)
    for cur_union in [au for au in all_unions_of_basis_sets if len(au) >= 2]:
        ## val = (val1, val2, val3, ... , val_k), where k = number of cur_unions sampled up to now.
        if doprint:
            print 'current union |------> %s' %(cur_union,)
    #     for val in all_values_of_unions:
    #         print 'cur val: --> %s' %(val,)
    #         for dx in range(get_min(cur_union,np.array(val)), get_max(cur_union,np.array(val))+1):
    #             print 'new val: --> %s' %(val+(dx,),)
        all_values_of_unions = [val+(dx,) for val in all_values_of_unions for dx in range(get_min(cur_union,np.array(val)), 1+get_max(cur_union,np.array(val)))]
#         if doprint:
#             for newv in all_values_of_unions:
#                 print 'new value: --> %s' %(newv,)
    print 'counted %d legal combos' %(len(all_values_of_unions))
    print 'so %f of combos are legal' %(float(len(all_values_of_unions))/total_possible_combos)
    return [dict(zip(all_unions_of_basis_sets, cc)) for cc in all_values_of_unions]

In [25]:
comb(4,2)+comb(4,3)+comb(4,4)

11.0

In [26]:
##test 
foo = enumerate_legal_combos(6,3)


total of 279936 possible combos
counted 2799 legal combos
so 0.009999 of combos are legal


In [27]:
##okay, this gives all possible combos
def enumerate_possible_counts(number_of_objects=1, number_of_basis_sets=2): ##enumerate all possible cardinalities of basis sets and their unions
    all_possible_unions_of_sets = nonemptysubsets(number_of_basis_sets)
    cnt = number_of_objects**len(all_possible_unions_of_sets)
    print cnt
    all_possible_values_of_unions = product(range(1,number_of_objects+1), repeat = len(all_possible_unions_of_sets))
    return [dict(zip(all_possible_unions_of_sets, cc)) for cc in all_possible_values_of_unions]
    
    

In [28]:
##compare all possible combos, to only the legal combos
adder = 0
noo = 2
num_basis = 2
all_counts = enumerate_possible_counts(number_of_objects=noo, number_of_basis_sets=num_basis)
for union_value_dict in all_counts:
    cnt = count_sets(target_index, union_value_dict,size_of_universe=noo)
    print '%s |----> %d' %(union_value_dict, cnt)
    adder += cnt
print 'total: %d' %(adder)

adder=0
all_counts = enumerate_legal_combos(number_of_objects=noo, number_of_basis_sets=num_basis)
for union_value_dict in all_counts:
    cnt = count_sets(target_index, union_value_dict,size_of_universe=noo)
    print '%s |----> %d' %(union_value_dict, cnt)
    adder += cnt
print 'total: %d' %(adder)

8
1
1
1
0
{(0, 1): 1, (0,): 1, (1,): 1} |----> 1
1
2
1
0
{(0, 1): 1, (0,): 2, (1,): 1} |----> 0
2
2
0
-1
{(0, 1): 1, (0,): 1, (1,): 2} |----> 0
2
3
0
-1
{(0, 1): 1, (0,): 2, (1,): 2} |----> 0
1
0
1
1
{(0, 1): 2, (0,): 1, (1,): 1} |----> 1
1
1
1
1
{(0, 1): 2, (0,): 2, (1,): 1} |----> 1
2
1
0
0
{(0, 1): 2, (0,): 1, (1,): 2} |----> 2
2
2
0
0
{(0, 1): 2, (0,): 2, (1,): 2} |----> 1
total: 6
total of 8 possible combos
counted 5 legal combos
so 0.625000 of combos are legal
1
1
1
0
{(0, 1): 1, (0,): 1, (1,): 1} |----> 1
1
0
1
1
{(0, 1): 2, (0,): 1, (1,): 1} |----> 1
1
1
1
1
{(0, 1): 2, (0,): 2, (1,): 1} |----> 1
2
1
0
0
{(0, 1): 2, (0,): 1, (1,): 2} |----> 2
2
2
0
0
{(0, 1): 2, (0,): 2, (1,): 2} |----> 1
total: 6


In [15]:
##for a basis with one set, fix the size of the the basis and target and then integrate over all sizes of their union
number_of_objects = 4
target_index = 0
basis_index = 1
target_size = 1
basis_size = 2  ##this is the number of elements in the lone basis set
all_counts = [{(target_index,): target_size, (basis_index,): basis_size, (target_index, basis_index): ii} for ii in range(1,number_of_objects+1)]

##should get comb(size_of_universe, size_of_target_set)
adder = 0
for union_value_dict in all_counts:
    cnt = count_sets(target_index, union_value_dict,size_of_universe=number_of_objects)
    print '%s |----> %d' %(union_value_dict, cnt)
    adder += cnt
print 'basis size: %d' %(basis_size)    
print 'total: %d' %(adder)
print 'should be: %d' %(comb(number_of_objects, target_size))

##value should not depend on "basis size" as long as <= number_of_objects
basis_size = 1
print '\n'
print 'new basis size: %d' %(basis_size)
all_counts = [{(target_index,): target_size, (basis_index,): basis_size, (target_index, basis_index): ii} for ii in range(1,number_of_objects+1)]
adder = 0
for union_value_dict in all_counts:
    cnt = count_sets(target_index, union_value_dict,size_of_universe=number_of_objects)
    print '%s |----> %d' %(union_value_dict, cnt)
    adder += cnt
print 'total: %d' %(adder)



{(0, 1): 1, (0,): 1, (1,): 2} |----> 0
{(0, 1): 2, (0,): 1, (1,): 2} |----> 2
{(0, 1): 3, (0,): 1, (1,): 2} |----> 2
{(0, 1): 4, (0,): 1, (1,): 2} |----> 0
basis size: 2
total: 4
should be: 4


new basis size: 1
{(0, 1): 1, (0,): 1, (1,): 1} |----> 1
{(0, 1): 2, (0,): 1, (1,): 1} |----> 3
{(0, 1): 3, (0,): 1, (1,): 1} |----> 0
{(0, 1): 4, (0,): 1, (1,): 1} |----> 0
total: 4


In [16]:
comb(3,0)

1.0

#### percentage of illegal configurations as a function of basis set size

In [17]:
##enumerate all possible cardinalities of basis sets and their unions
target_index = 0
max_no = 10
max_bs = 5
cnt = 0
results_dict = {'n_objects': [], 'n_basis': [], 'legal': [], 'not_legal': [], 'count_dist': []}
for number_of_objects in range(2, max_no+1):
    legal_cnt = 0
    non_legal_cnt = 0
    for number_of_basis_sets in range(2, max_bs+1):
        all_counts = enumerate_possible_coounts(number_of_objects=number_of_objects, number_of_basis_sets=number_of_basis_sets)
        for union_value_dict in all_counts:
            cnt = count_sets(target_index, union_value_dict,size_of_universe=number_of_objects)
            if cnt:
                legal_cnt += 1
            else:
                non_legal_cnt += 1
        results_dict['n_objects'].append(number_of_objects)
        results_dict['n_basis'].append(number_of_basis_sets)
        results_dict['legal'].append(legal_cnt)
        results_dict['not_legal'].append(non_legal_cnt)

print results_dict

NameError: name 'enumerate_possible_coounts' is not defined

In [ ]:
import pandas as pd

In [ ]:
foo = pd.DataFrame(results_dict)

In [ ]:
686./77564

In [ ]:
baz = foo[['legal', 'not_legal']].groupby(foo['n_basis'])


In [ ]:
baz.head()